## Struktury danych

In [28]:
class Point:
    def __init__(self,x,y):
        self.x=x
        self.y=y

In [29]:
class Segment:
    def __init__(self,p,q):
        
        if p.x < q.x:
            left, right=p, q
        else:
            left, right=q, p
                    
        self.left=left
        self.right=right
        self.a=(right.y-left.y)/(right.x-left.x) # zakladam ze nie ma pionowych odcinkow
        self.b=left.y-left.x*self.a
        
    def get_y(self,x):
        return self.a*x+self.b


In [30]:
class Trapezoid:
    def __init__(self, top, bottom, leftp, rightp):

        self.top = top
        self.bottom = bottom
        self.leftp = leftp
        self.rightp = rightp

        # wskazniki do sasiadow
        self.upper_left = None
        self.lower_left = None
        self.upper_right = None
        self.lower_right = None


In [31]:
class DNode:
    def __init__(self, node_type, label, left=None, right=None):
        """
        node_type: Typ węzła ('x', 'y' lub 'leaf')
        label: Punkt, odcinek lub trapez, zależnie od typu węzła
        left: Lewy potomek
        right: Prawy potomek
        """
        self.node_type = node_type  # Typ węzła: 'x', 'y', 'leaf'
        self.label = label          # Punkt, odcinek lub trapez
        self.left = left            # Lewy potomek
        self.right = right          # Prawy potomek

    
class DTree:
    def __init__(self):
        self.root = None

    def query(self, point,a=None):
        """
        Znajduje trapez, który zawiera dany punkt.
        point: (x, y) - współrzędne punktu
        Zwraca: Trapez (etykieta liścia), który zawiera punkt
        """
        current = self.root
        while current:
            if current.node_type == 'x':
                # Testujemy po współrzędnej x
                if point.x < current.label.x:
                    current = current.left                    
                else:
                    current = current.right

            elif current.node_type == 'y':
                # Testujemy położenie względem odcinka
                segment = current.label
                y_on_line = segment.get_y(point.x)
                if point.y > y_on_line:
                    current = current.left
                elif point.y==y_on_line and a is not None:   # potrzebne do wyznaczania strefy dla s_i, jeżeli początek dodawanego odcinka leży na sprawdzanej linii
                    # nie wiem czy to rozwiazuje problem  
                    if segment.a>a:
                        current = current.right
                    else:
                        current = current.left
                else:
                    current = current.right

            elif current.node_type == 'leaf':
                # Znaleziony trapez
                return current.label

        return None

# Na wykładzie są podane dwa rodzaje tej struktury - musimy wytłumaczyć dlaczego akurat tą wybierami?

In [32]:
# Testy struktury DTree
if __name__ == "__main__":
    # Tworzenie przykładowych danych
    p1 = Point(1, 1)
    p2 = Point(5, 5)
    p3 = Point(3, 2)
    segment = Segment(p1, p2)

    trapez1 = Trapezoid(None, None, p1, p2)
    trapez2 = Trapezoid(None, None, p2, p3)

    leaf1 = DNode('leaf', trapez1)
    leaf2 = DNode('leaf', trapez2)
    x_node = DNode('x', p2, leaf1, leaf2)

    # Tworzenie drzewa
    dtree = DTree()
    dtree.root = x_node

    # Test punktu
    test_point = Point(4, 4)
    result = dtree.query(test_point)
    print("Trapez zawierający punkt:", result.leftp.x, result.rightp.x)

Trapez zawierający punkt: 1 5


## Aplikacja do zadawania odcinków

Aby wprowadzić odcinek należy nacisnąć lewym przyciskiem myszy w punkcie początkowym, przeciągnąć kursor i zwolnić przycisk w punkcie końcowym.
Po wprowadzeniu wszystkich odcinków wystarczy zamknąć okno.

In [37]:
%matplotlib tk

import matplotlib.pyplot as plt

class LineDrawer:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.lines = []
        self.current_line = None
        self.start_point = None

        # stałe wymiary okna (można zmienić)
        self.ax.set_xlim(0, 10)
        self.ax.set_ylim(0, 10)

        self.cid_click = self.fig.canvas.mpl_connect('button_press_event', self.on_click)
        self.cid_motion = self.fig.canvas.mpl_connect('motion_notify_event', self.on_motion)
        self.cid_release = self.fig.canvas.mpl_connect('button_release_event', self.on_release)

    def on_click(self, event):
        if event.inaxes != self.ax:
            return
        self.start_point = (event.xdata, event.ydata)
        self.ax.plot(event.xdata, event.ydata, 'ko') 
        self.current_line = self.ax.plot([event.xdata, event.xdata], [event.ydata, event.ydata], 'k-')[0]

    def on_motion(self, event):
        if self.current_line is None or event.inaxes != self.ax:
            return
        xdata, ydata = self.current_line.get_data()
        xdata[1], ydata[1] = event.xdata, event.ydata
        self.current_line.set_data(xdata, ydata)
        self.fig.canvas.draw()

    def on_release(self, event):
        if self.current_line is None or event.inaxes != self.ax:
            return
        self.lines.append((self.start_point, (event.xdata, event.ydata)))
        self.ax.plot(event.xdata, event.ydata, 'ko') 
        self.current_line = None
        self.start_point = None
        self.fig.canvas.draw()

    def show(self):
        plt.show()

    def get_lines(self):
        return self.lines

drawer = LineDrawer()
drawer.show()
lines=drawer.get_lines()


In [38]:
print(lines)

[((np.float64(1.8548387096774195), np.float64(7.7272727272727275)), (np.float64(6.629032258064516), np.float64(8.54978354978355))), ((np.float64(4.532258064516128), np.float64(3.7445887445887447)), (np.float64(8.516129032258064), np.float64(0.9740259740259745))), ((np.float64(1.1451612903225805), np.float64(2.0346320346320352)), (np.float64(4.032258064516128), np.float64(5.670995670995671)))]


In [39]:
# zamiana reprezentacji odcinków
for i in range(len(lines)):
    p=Point(lines[i][0][0],lines[i][0][1])
    q=Point(lines[i][1][0],lines[i][1][1])
    segment=Segment(p,q)
    lines[i]=segment

## Konstrukcja mapy trapezowej T(S)

wyznaczanie strefy dla $s_i$

In [36]:
def find_area_to_delete(D,segment):
    area=set()
    p=segment.left
    q=segment.right
    delta=D.query(p)
    area.add(delta)
    while q.x>delta.rightp.x:
        if delta.rightp.y>segment.get_y(delta.rightp.x):
            area.add(delta.lower_right)
            delta=delta.lower_right
        else:
            area.add(delta.upper_right)
            delta=delta.upper_right
    return area
        

